## Collecting Zip codes and mapping them to Lat and Lon

The zillow city dataset does not follow the same naming convention as the Census data. About 30% of the zillow data does not map on the city names of the census data. That being said, we have the lat. and long. for each city in the census data and zillow offers a very comprehensive dataset for housing prices based on zipcodes.

The scope of this notebook is to map Latitude and Longidute to each zipcode in the zillow dataset. We are using a geo zip code data with 43000 zip codes to map to the zillow data. There are about 220 zipcodes in the zillow dataset that do not have a match in the geo zip code dataset. For these remaining zipcodes we will be pulling data from the smartystreets.com api.

In [1]:
import pandas as pd

In [2]:
import os

path = 'data/housing/bedroomzip/'
files = os.listdir(path)
names = [i.split('.csv')[0] for i in files]
names

['Zip_Zhvi_1bedroom',
 'Zip_Zhvi_2bedroom',
 'Zip_Zhvi_3bedroom',
 'Zip_Zhvi_4bedroom',
 'Zip_Zhvi_5Bedroom']

In [3]:
for i in files:
    name = i.split('.csv')[0]
    vars()[name] = pd.read_csv(path + i, encoding='latin')
    print(vars()[name].shape)

(13607, 150)
(21864, 150)
(24076, 150)
(24071, 150)
(20184, 150)


In [5]:
Zip_Zhvi_3bedroom.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,2008-01,2008-02,2008-03,...,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,NaN,NaN,NaN,...,2.071432e+06,2.056974e+06,2.042780e+06,2.035366e+06,2.034423e+06,2.025748e+06,2.005972e+06,1.973243e+06,1.958949e+06,1.948357e+06
1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,NaN,NaN,NaN,...,5.894837e+05,5.897887e+05,5.917907e+05,5.927763e+05,5.910983e+05,5.901127e+05,5.871780e+05,5.847233e+05,5.823167e+05,5.815510e+05
2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,NaN,NaN,NaN,...,3.483464e+06,3.445784e+06,3.422120e+06,3.384821e+06,3.368683e+06,3.347543e+06,3.338804e+06,3.334868e+06,3.346086e+06,3.355841e+06
3,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,197135.0,196620.0,196120.333333,...,2.545590e+05,2.549057e+05,2.540797e+05,2.533590e+05,2.528003e+05,2.530083e+05,2.531913e+05,2.541217e+05,2.546267e+05,2.555150e+05
4,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,5,NaN,NaN,NaN,...,6.942263e+05,6.947930e+05,6.981180e+05,6.997963e+05,6.988157e+05,6.963590e+05,6.934267e+05,6.905770e+05,6.891443e+05,6.878177e+05


In [8]:
Zip_Zhvi_3bedroom['2008-01'].mean()

205668.97360857067

In [9]:
df = pd.read_csv('data/housing/us-zip-code-latitude-and-longitude.csv', sep=';')

In [14]:
needed = []
num = 0
for i in names:
    out = vars()[i]
    for a in out.RegionName.values:
        if a not in check and a not in needed:
            num += 1
            needed.append(a)
len(needed)

214

In [16]:
import requests

get = requests.get(f"https://us-zipcode.api.smartystreets.com/lookup?auth-id=ffe8db69-6b00-f03d-b5fe-3d39699890ce&auth-token=PQ6B8aOVVhpKgsTcSomX&zipcode=6461")

In [24]:
format_it = get.json()[0]['zipcodes'][0]
lat = format_it['latitude']
lng = format_it['longitude']

In [33]:
def get_data(zipc):
    try:
        get = requests.get(f"https://us-zipcode.api.smartystreets.com/lookup?auth-id=ffe8db69-6b00-f03d-b5fe-3d39699890ce&auth-token=PQ6B8aOVVhpKgsTcSomX&zipcode={str(zipc)}")
        format_it = get.json()[0]['zipcodes'][0]
        lat = format_it['latitude']
        lng = format_it['longitude']
        res = (zipc, lat, lng)
    except:
        res = (zipc, False, False)
    return(res)

In [34]:
it = get_data(39826)

In [35]:
it[1]

31.82074

In [36]:
import time
import concurrent.futures

In [38]:
t1 = time.perf_counter()

with concurrent.futures.ThreadPoolExecutor() as executor:
    export_data = executor.map(get_data, needed)


t2 = time.perf_counter()
print(f'Finished in {t2-t1} seconds')

Finished in 1.766184100000146 seconds


In [39]:
loop_data = [i for i in export_data]

In [48]:
remaining_zips = {}
for i in loop_data:
    remaining_zips[int(i[0])] = {'lat':i[1], 'lng':i[2]}

In [43]:
remaining_zips[85142]

{'lat': 33.23384, 'lng': -111.64473}

In [65]:
main_dic = {}
for i in df[['Zip','Latitude','Longitude']].values:
    main_dic[i[0]] = {'lat':i[1], 'lng':i[2]}

In [66]:
lat_place = []
long_place = []
for i in Zip_Zhvi_1bedroom.RegionName.values:
    try:
        lat_place.append(main_dic[i]['lat'])
        long_place.append(main_dic[i]['lng'])
    except:
        lat_place.append(remaining_zips[i]['lat'])
        long_place.append(remaining_zips[i]['lng'])     
    

In [67]:
len(lat_place)

13607

In [68]:
len(Zip_Zhvi_1bedroom)

13607

In [70]:
for a in names:
    lat_place = []
    long_place = []
    for i in vars()[a].RegionName.values:
        try:
            lat_place.append(main_dic[i]['lat'])
            long_place.append(main_dic[i]['lng'])
        except:
            lat_place.append(remaining_zips[i]['lat'])
            long_place.append(remaining_zips[i]['lng']) 
    vars()[a]['lat'] = lat_place
    vars()[a]['lng'] = long_place

In [71]:
names

['Zip_Zhvi_1bedroom',
 'Zip_Zhvi_2bedroom',
 'Zip_Zhvi_3bedroom',
 'Zip_Zhvi_4bedroom',
 'Zip_Zhvi_5Bedroom']

In [72]:
Zip_Zhvi_4bedroom.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,2008-01,2008-02,2008-03,...,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,lat,lng
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,NaN,NaN,NaN,...,2998417.0,3.008020e+06,3.018742e+06,3.001803e+06,2.983767e+06,2.962636e+06,2.980488e+06,3.002555e+06,40.798502,-73.96811
1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,NaN,NaN,NaN,...,1010288.0,1.012454e+06,1.010338e+06,1.008764e+06,1.006188e+06,1.006068e+06,1.005388e+06,1.005377e+06,41.940832,-87.65852
2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,NaN,NaN,NaN,...,6367316.0,6.409420e+06,6.425129e+06,6.446475e+06,6.487110e+06,6.514059e+06,6.543494e+06,6.524043e+06,40.776099,-73.98285
3,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,267042.0,266167.5,265567.333333,...,331012.0,3.302250e+05,3.295703e+05,3.297790e+05,3.298797e+05,3.309300e+05,3.314603e+05,3.318457e+05,29.760833,-95.81104
4,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,5,NaN,NaN,NaN,...,1253091.0,1.256735e+06,1.252838e+06,1.248100e+06,1.242790e+06,1.239398e+06,1.237450e+06,1.234638e+06,41.922682,-87.65432


In [78]:
new_path = 'data/housing/bedroomziplatlng/'
for i in names:
    vars()[i].to_csv(new_path + i + '_zip'+ '.csv')

In [79]:
os.listdir(new_path)

['Zip_Zhvi_1bedroom_zip.csv',
 'Zip_Zhvi_2bedroom_zip.csv',
 'Zip_Zhvi_3bedroom_zip.csv',
 'Zip_Zhvi_4bedroom_zip.csv',
 'Zip_Zhvi_5Bedroom_zip.csv']

In [80]:
checkdf = pd.read_csv(new_path + names[0] + '_zip' + '.csv')

In [81]:
checkdf.head()

,Unnamed: 0,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,2008-01,2008-02,...,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,lat,lng
0,0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,NaN,NaN,...,724675.000000,723230.333333,720543.000000,716854.333333,714189.000000,710429.666667,709241.333333,707854.666667,40.798502,-73.96811
1,1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,NaN,NaN,...,217307.666667,217699.333333,217167.000000,216393.333333,214804.666667,213714.666667,212630.666667,212261.666667,41.940832,-87.65852
2,2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,NaN,NaN,...,826669.333333,825952.333333,822924.000000,821563.666667,818487.666667,811886.666667,806982.333333,803770.333333,40.776099,-73.98285
3,3,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,4,NaN,NaN,...,256729.666667,255682.333333,254573.333333,253914.666667,253288.000000,252864.666667,252103.666667,251404.666667,41.922682,-87.65432
4,4,91940,77449,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,5,120884.0,120536.5,...,147189.333333,147436.333333,147741.666667,147999.666667,148357.000000,148418.666667,148552.666667,148807.000000,29.825908,-95.73010
